 **Source template:** <a href="https://colab.research.google.com/github/johanhoffman/DD2365_VT21/blob/main/template-report-Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Stokes equations**
**Johan Hoffman**

**Lab made by Miguel De Le Court**

# **Abstract**
In this short report, we use FEniCS to solve the Stokes equations. We verify that Taylor-Hood elements are stable, while equal order elements for the velocity and pressure lead to unstable results. We also implement an stable equal order simulation using the Brezzi-Pitkäranta stabilization.

This report was made as part of the course DD2365 Advanced Computation in Fluid Mechanics, at the KTH Royal Institute of Technology.

[DD2365 course website.](https://kth.instructure.com/courses/17071)

# **About the code**

In [ ]:
"""This program is based on an example file for the course"""
"""DD2365 Advanced Computation in Fluid Mechanics, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""
"""The template file is available at https://github.com/johanhoffman/DD2365_VT21"""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Introduction**
The Stokes equations take the form

$$\nabla p -\Delta u = f,\quad \nabla \cdot u=0$$

together with suitable boundary conditions.

Here we present a FEniCS implementation of a mixed finite element method to solve the Stokes equations in 2D. The solution is visualized using FEniCS plotting functions, and is also exported as pvd-files which can be visualized in Paraview.

To derive the weak form of the equations, multiply the momentum equation by $v\in V$ and the continuity equation by $q\in Q$, and then integrate over the domain $\Omega$ and use Green's formula
$$
(\nabla p -\Delta u,v) = -(p,\nabla \cdot v) + (\nabla u, \nabla v) + <p n - \nabla u\cdot n, v>_{\Gamma}
$$

We seek a finite element approximation $(u,p)\in V\times Q$ such that 

$$
- (p,\nabla \cdot v) + (\nabla u,\nabla v) + (\nabla \cdot u, q) + <pn - \nabla u\cdot n, v>_{\partial \Omega} = (f,v)
$$

for all test functions $(v,q) \in V\times Q$. 

$$
(v,w) = \int_{\Omega} v\cdot w ~dx, \quad 
<v,w>_{\partial \Omega} = \int_{\partial \Omega} v\cdot w~ds
$$

We divide the boundary into $\partial \Omega=\Gamma_D \cup \Gamma_N$, with the different boundary conditions

$$
u = g_D,\quad x\in \Gamma_D,
$$

$$
-\nu \nabla u\cdot n + pn = g_N, \quad x\in \Gamma_N,
$$

For $x\in \Gamma_D$ the test function $v=0$. Hence, with $g_N=0$, as is the case here, the boundary term is zero. 

The equations can be expressed in residual form

$$
r(u,p;v,q) = - (p,\nabla \cdot v) + (\nabla u,\nabla v) + (\nabla \cdot u, q) - (f,v)
$$

To implement the velocity boundary conditions we use a penalty formulation, with a penalty parameter $\gamma = C/h$, with $C>0$ a constant and $h$ the local mesh size. At outflow we use a "do nothing" stress free boundary condition. 


# **Set up environment**
## **Packages**

In [ ]:
# Load neccessary modules.
try:
    from google.colab import files #Unnecessary if ran locally
    ! sudo apt-get install texlive-latex-extra 
    ! sudo apt install texlive-fonts-recommended
    ! sudo apt install dvipng
    ! sudo apt-get install cm-super
except:
    pass

import numpy as np
import time

# Install FEniCS
try:
    import dolfin
except ImportError as e:
    !apt-get install -y -qq software-properties-common
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    !sed -i "s|#if PETSC_VERSION_MAJOR == 3 && PETSC_VERSION_MINOR <= 8 && PETSC_VERSION_RELEASE == 1|#if 1|" /usr/include/dolfin/la/PETScLUSolver.h
    !rm -rf /usr/lib/python3/dist-packages/mpi4py*
    !rm -rf /usr/lib/python3/dist-packages/petsc4py*
    !rm -rf /usr/lib/python3/dist-packages/slepc4py*
    !rm -rf /usr/lib/petsc/lib/python3/dist-packages/dolfin*
    !rm -rf /usr/lib/petsc/lib/python3/dist-packages/mshr*
    !wget "https://drive.google.com/uc?export=download&id=1cT_QBJCOW_eL3BThnval3bcpb8o0w-Ad" -O /tmp/mpi4py-2.0.0-cp37-cp37m-linux_x86_64.whl
    !wget "https://drive.google.com/uc?export=download&id=119i49bxlGn1mrnhTNmOvM4BqmjrT9Ppr" -O /tmp/petsc4py-3.7.0-cp37-cp37m-linux_x86_64.whl
    !wget "https://drive.google.com/uc?export=download&id=1-1tVfu8qz3bRC2zvR8n3RESpesWqNnn6" -O /tmp/slepc4py-3.7.0-cp37-cp37m-linux_x86_64.whl
    !wget "https://drive.google.com/uc?export=download&id=1-3qY4VIJQaXVO1HfGQIzTIURIeJbvX-9" -O /tmp/fenics_dolfin-2019.2.0.dev0-cp37-cp37m-linux_x86_64.whl
    !wget "https://drive.google.com/uc?export=download&id=1-5SMjgjMuee_9WLeYtGe8N_lvipWEN7W" -O /tmp/mshr-2019.2.0.dev0-cp37-cp37m-linux_x86_64.whl
    !pip3 install /tmp/mpi4py-2.0.0-cp37-cp37m-linux_x86_64.whl --upgrade
    !pip3 install /tmp/petsc4py-3.7.0-cp37-cp37m-linux_x86_64.whl --upgrade
    !pip3 install /tmp/slepc4py-3.7.0-cp37-cp37m-linux_x86_64.whl --upgrade
    !pip3 install /tmp/fenics_dolfin-2019.2.0.dev0-cp37-cp37m-linux_x86_64.whl --upgrade
    !pip3 install /tmp/mshr-2019.2.0.dev0-cp37-cp37m-linux_x86_64.whl --upgrade
    !pip3 -q install --upgrade sympy
    import dolfin

from dolfin import *; from mshr import *

import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt

In [ ]:
# Setting up matplotlib to use TeX.
import matplotlib
matplotlib.rc("text", usetex=True)
matplotlib.rc("font", family="serif")
matplotlib.rc('image', cmap="Spectral_r")
usingtex = True
matplotlib.rcParams.update({'font.size': 14})

## **Mesh**
Since we will use the same mesh for all the computations, we only define it once below.

In [ ]:
# Define rectangular domain 
L = 4
H = 2
r = 0.2
fsize=3 #Define the size of the figures relative to the size of the domain

# Generate mesh (examples with and without a hole in the mesh) 
resolution = 32
#mesh = RectangleMesh(Point(0.0, 0.0), Point(L, H), L*resolution, H*resolution)
#mesh = generate_mesh(Rectangle(Point(0.0,0.0), Point(L,H)) - Circle(Point(0.5,0.5*H),0.2), resolution)
mesh = generate_mesh(Rectangle(Point(0.0,0.0), Point(L,H)) - Circle(Point(L/2,H/2),r), resolution)

# Local mesh refinement (specified by a cell marker)
no_levels = 1
for i in range(0,no_levels):
  cell_marker = MeshFunction("bool", mesh, mesh.topology().dim())
  for cell in cells(mesh):
    cell_marker[cell] = False
    p = cell.midpoint()
    if p.distance(Point(L/2, H/2)) < r + (1-r)/(4**i):
        cell_marker[cell] = True
  mesh = refine(mesh, cell_marker)

fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, title=r"\textbf{Mesh}")
plt.xlim((0,L)); plt.ylim((0,H))
plt.show()

# Inf-sup condition: Taylor-hood elements

We first define the finite element approximation spaces as taylor-hood elements, with degree 2 for the velocity and 1 for the pressure.

In [ ]:
# Generate mixed finite element spaces (for velocity and pressure)
VE = VectorElement("CG", mesh.ufl_cell(), 2)
QE = FiniteElement("CG", mesh.ufl_cell(), 1)
WE = VE * QE

W = FunctionSpace(mesh, WE)
V = FunctionSpace(mesh, VE)
Q = FunctionSpace(mesh, QE)

# Define trial and test functions
w = Function(W)
(u, p) = (as_vector((w[0],w[1])), w[2])
(v, q) = TestFunctions(W) 

**Then define boundary conditions**

In [ ]:
# Examples of inflow and outflow conditions
XMIN = 0.0; XMAX = L
YMIN = 0.0; YMAX = H
uin = Expression(("4*(x[1]*(YMAX-x[1]))/(YMAX*YMAX)", "0."), YMAX=YMAX, element = V.ufl_element()) 
#pout = 0.0

# Inflow boundary (ib), outflow boundary (ob) and wall boundary (wb)
ib = Expression("near(x[0],XMIN) ? 1. : 0.", XMIN=XMIN, element = Q.ufl_element())
ob = Expression("near(x[0],XMAX) ? 1. : 0.", XMAX=XMAX, element = Q.ufl_element()) 
wb = Expression("x[0] > XMIN + DOLFIN_EPS && x[0] < XMAX - DOLFIN_EPS ? 1. : 0.", XMIN=XMIN, XMAX=XMAX, element = Q.ufl_element())

**And define and solve variational problem**

In [ ]:
h = CellDiameter(mesh)
C = 1.0e3
gamma = C/h

f = Expression(("0.0","0.0"), element = V.ufl_element())

# Define variational problem on residual form: r(u,p;v,q) = 0
residual = ( - p*div(v)*dx + inner(grad(u), grad(v))*dx + div(u)*q*dx + 
            gamma*(ib*inner(u - uin, v) + wb*inner(u, v))*ds - inner(f, v)*dx )

# Solve algebraic system 
solve(residual == 0, w) 

## **Results**
The velocity field is shown below. As expected, no instabilities seem to appear.

In [ ]:
!rm results-NS/*

# Open files to export solution to Paraview
file_u = File("results-Stokes/u.pvd")
file_p = File("results-Etokes/p.pvd")

u1 = project(u, V)
p1 = project(p, Q)

# Save solution to file
file_u << u1
file_p << p1

# Plot solution
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(u1, title=r"\textbf{Velocity field (CG2/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()


# Export files
#!tar -czvf results-Stokes.tar.gz results-NS
#files.download('results-Stokes.tar.gz')

The same can be said for the pressure field below, verifying that the Taylor-Hood elements are indeed stable.


In [ ]:
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(p1, title=r"\textbf{Pressure field (CG2/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()


# Inf-sup condition: Equal order spaces
We now recompute everything with first-order basis functions for the velocity and pressure.


In [ ]:
u_th = u
p_th = p

# Generate mixed finite element spaces (for velocity and pressure)
VE = VectorElement("CG", mesh.ufl_cell(), 1)
QE = FiniteElement("CG", mesh.ufl_cell(), 1)
WE = VE * QE

W = FunctionSpace(mesh, WE)
V = FunctionSpace(mesh, VE)
Q = FunctionSpace(mesh, QE)



# Define trial and test functions
w = Function(W)
(u, p) = (as_vector((w[0],w[1])), w[2])
(v, q) = TestFunctions(W) 

In [ ]:
# Examples of inflow and outflow conditions
XMIN = 0.0; XMAX = L
YMIN = 0.0; YMAX = H
uin = Expression(("4*(x[1]*(YMAX-x[1]))/(YMAX*YMAX)", "0."), YMAX=YMAX, element = V.ufl_element()) 
#pout = 0.0

# Inflow boundary (ib), outflow boundary (ob) and wall boundary (wb)
ib = Expression("near(x[0],XMIN) ? 1. : 0.", XMIN=XMIN, element = Q.ufl_element())
ob = Expression("near(x[0],XMAX) ? 1. : 0.", XMAX=XMAX, element = Q.ufl_element()) 
wb = Expression("x[0] > XMIN + DOLFIN_EPS && x[0] < XMAX - DOLFIN_EPS ? 1. : 0.", XMIN=XMIN, XMAX=XMAX, element = Q.ufl_element())

In [ ]:
h = CellDiameter(mesh)
C = 1.0e3
gamma = C/h

f = Expression(("0.0","0.0"), element = V.ufl_element())

# Define variational problem on residual form: r(u,p;v,q) = 0
residual = ( - p*div(v)*dx + inner(grad(u), grad(v))*dx + div(u)*q*dx  + gamma*(ib*inner(u - uin, v) + wb*inner(u, v))*ds - inner(f, v)*dx) #+ 1e-2*h**2*inner(grad(p), grad(q))*dx

# Solve algebraic system 
solve(residual == 0, w) 

In [ ]:
!rm results-NS/*

# Open files to export solution to Paraview
file_u = File("results-Stokes/u.pvd")
file_p = File("results-Etokes/p.pvd")

u1 = project(u, V)
p1 = project(p, Q)

# Save solution to file
file_u << u1
file_p << p1

# Plot solution
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(u1, title=r"\textbf{Velocity field (CG1/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

# Export files
#!tar -czvf results-Stokes.tar.gz results-NS
#files.download('results-Stokes.tar.gz')

## **Results**

While at first glance the velocity field shown above seems reasonable, the pressure shown below reveals the instabilities. Those manifest mainly around the center hole as high frequency perturbations, resulting in an arbitrarily large pressure gradient.

In [ ]:
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(p1, title=r"\textbf{Pressure field (CG1/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()


This is confirmed, as can be seen on the figures below. The pressure gradient computed with CG1/CG1 elements goes to more than 4000 here, and this value increases with the mesh refinement. Note that when the mesh is refined locally around the center hole, the boundary doesn't change, meaning that sharp angles remain. This non-smooth boundary causes large gradients in the pressure for both methods. The difference is that these large gradients do not appear with CG2/CG1 it the boundary is smooth. The pressure gradient from the previous solution (CG2/CG1) remains bounded at around 80 if there is no *local* mesh refinement, and at around 500 when we *locally* refine the mesh once.


In [ ]:
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(inner(grad(p1), grad(p1))**0.5, title=r"\textbf{Norm of the pressure gradient (CG1/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(inner(grad(p_th), grad(p_th))**0.5, title=r"\textbf{Norm of the pressure gradient (CG2/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

# **Equal order spaces with stabilization**
We now introduce the Brezzi-Pitkäranta stabilization in the CG1-CG1 model. The only change is an added term in the weak form of the equation:

$$
r(u,p;v,q) = - (p,\nabla \cdot v) + (\nabla u,\nabla v) + (\nabla \cdot u, q) + C h^2( \nabla p, \nabla q ) - (f,v)
$$

where $C$ is a constant and $h$ is the local element size. 


In [ ]:
u_11 = u
p_11 = p
w = Function(W)
(u, p) = (as_vector((w[0],w[1])), w[2])
(v, q) = TestFunctions(W) 

h = CellDiameter(mesh)
C = 1.0e3
gamma = C/h

f = Expression(("0.0","0.0"), element = V.ufl_element())

# Define variational problem on residual form: r(u,p;v,q) = 0
residual = ( - p*div(v)*dx + inner(grad(u), grad(v))*dx + div(u)*q*dx  + gamma*(ib*inner(u - uin, v) + wb*inner(u, v))*ds - inner(f, v)*dx + 1e-1*h**2*inner(grad(p), grad(q))*dx)

# Solve algebraic system 
solve(residual == 0, w) 

In [ ]:
!rm results-NS/*

# Open files to export solution to Paraview
file_u = File("results-Stokes/u.pvd")
file_p = File("results-Etokes/p.pvd")

u1 = project(u, V)
p1 = project(p, Q)

# Save solution to file
file_u << u1
file_p << p1

# Plot solution (velocity is not very interesting)
"""
fig = plt.figure(figsize=(H*fsize,L*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(u1, title="Velocity field (CG1/CG1)")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()
"""

fig = plt.figure(figsize=(L*fsize, H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(p1, title=r"\textbf{Pressure field (CG1/CG1) + stabilization}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)


fig = plt.figure(figsize=(L*fsize, H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(p_th, title=r"\textbf{Pressure field (CG2/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()
# Export files
#!tar -czvf results-Stokes.tar.gz results-NS
#files.download('results-Stokes.tar.gz')

As can be seen above, Brezzi-Pitkäranta stabilization removes the spurious oscillations in the  pressure. It is worth nothing that the behaviour of the stabilization depends a lot on the value of the constant $C$. Here we set $C$ to $0.1$. A larger value tends to excessively smooth the pressure, while a value too small ($<<0.01$ here) is barely different from the non-stabilised simuation. A larger value of $C$ also mitigates the effects of the sharp corners on the pressure gradient, as can be seen below.

In [ ]:
fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(inner(grad(p1), grad(p1))**0.5, title=r"\textbf{Norm of the pressure gradient (CG1/CG1) + stabilization}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()

fig = plt.figure(figsize=(L*fsize,H*fsize))
plot(mesh, color="k", linewidth=0.1, alpha=0.7)
plt.xlim((0,L)); plt.ylim((0,H))
im = plot(inner(grad(p_th), grad(p_th))**0.5, title=r"\textbf{Norm of the pressure gradient (CG2/CG1)}")
ax = plt.gca()
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(im, cax=cax)
plt.show()


# **Discussion**

In conclusion, we observed that a Taylor-Hood mixed finite element method was was stable to solve the Stokes equations in 2D. By contrast, equal order elements for the pressure and velocity result in an instalble method and spurious oscillations in the pressure. An equal order interpolation is possible if a stabilization term is added such as the Brezzi-Pitkäranta stabilization. All the methods were tested for the model problem of flow past a number of circular obstacles, and the solution behaved as expected.  